In [18]:
!pip3 install pandas bs4 geopy

     |████████████████████████████████| 102kB 1.2MB/s ta 0:00:011


In [2]:
import json
import os

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
!which ipython3

/usr/local/bin/ipython3


In [4]:
url_path = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
css_selector = "table.wikitable.sortable.jquery-tablesorter"

In [5]:
page = requests.get(url_path)
soup = BeautifulSoup(page.text, 'html.parser')

In [6]:
my_table = soup.find('table', attrs={'class': 'wikitable sortable'})

In [7]:
headers = "Postcode,Borough,Neighbourhood"
table1 = ""
for tr in my_table.find_all('tr'):
    row1 = ""
    for tds in tr.find_all('td'):
        row1 = row1 + "," + tds.text
    table1 = table1 + row1[1:]

In [8]:
file = open("toronto.csv", "wb")
file.write(bytes(table1, encoding="ascii", errors="ignore"))
file.close()

In [9]:
import pandas as pd
df = pd.read_csv('toronto.csv', header=None)
df.columns = ["Postalcode", "Borough", "Neighbourhood"]

In [10]:
indexNames = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNames, inplace=True)

In [11]:
result = df.groupby(['Postalcode', 'Borough'], sort=False).agg(', '.join)

In [12]:
df_new = result.reset_index()
df_new.head(15)

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Not assigned
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [13]:
df_new.shape

(103, 3)

In [14]:
df_lon_lat = pd.read_csv('Geospatial_Coordinates.csv')
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_lon_lat.columns=['Postalcode','Latitude','Longitude']
Toronto_df = pd.merge(df_new,
                 df_lon_lat[['Postalcode','Latitude', 'Longitude']],
                 on='Postalcode')
Toronto_df

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
